In [ ]:
from Environment.custom_env import CustomEnvironment
import json
import torch
import numpy as np
from tqdm._tqdm_notebook import tqdm_notebook

In [ ]:
torch.cuda.is_available()

In [ ]:
f = open('Environment/env_config.json')
env_variables = json.load(f)
f.close

In [ ]:
env = CustomEnvironment(env_config=env_variables, render_mode="human")
# env = parallel_env(render_mode="human")
# env.reset(seed=42)

In [ ]:
import matplotlib.pyplot as plt 
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler(feature_range=(0, 128))

In [ ]:
episodes = 100

for episode in tqdm_notebook(range(episodes)): #episodes loop"

    curr_obs,infos = env.reset() #reset environment after each episode\n",

    losses = {}
    while(env.agents): #until there are any surviving agents 

        actions = {}
        log_probs = {}

        for i in range(env.n_agents): #objects contains all of the models, get the corresponding actions from each policy\n",
            
            if i in env.agents:
                act, log_prob = env.agents_objects[i].brain1.select_action(state = torch.tensor(curr_obs[i], dtype = torch.float32))
                actions[i] = act
                log_probs[i] = log_prob
            else:
                actions[i] = env.num_actions
                log_probs[i] = 69
            
        # print(actions)

        next_obs, rewards, terminations, truncations, infos = env.step(actions)
        
        # print(env.agents_objects)
        # print(curr_obs)
        # print(rewards)
        # print(terminations)
        # print(truncations)
        # print(infos)
        
        for i in range(env.n_agents):

            if i in env.agents:
                env.agents_objects[i].brain1.push_to_buffer(torch.tensor(curr_obs[i], dtype = torch.float32),actions[i],rewards[i],torch.tensor(next_obs[i], dtype = torch.float32),log_probs[i],terminations[i])
                env.agents_objects[i].brain1.update_weights() 
            
            if i in env.justdie:
                env.justdie[i].brain1.push_to_buffer(torch.tensor(curr_obs[i], dtype = torch.float32),actions[i],rewards[i],torch.tensor(next_obs[i], dtype = torch.float32),log_probs[i],terminations[i])
                loss = env.justdie[i].brain1.update_weights() 
                del env.justdie[i]
                losses[i] = loss
        
        curr_obs = next_obs

    print(f"Episode {episode} end")
    print(f"Losses {losses}")
    print()
    env.close()